In [1]:
import pandas as pd
import numpy as np
import scipy.io
from sklearn import metrics

In [2]:
# load mat dataset
def load_dataset(filepath):
    mat = scipy.io.loadmat(filepath)
    df = pd.DataFrame(np.hstack((mat['X'], mat['y'])))
    y_col = df.shape[1] - 1
    X, y = df.drop(y_col, axis=1).to_numpy(), df[y_col]
    return X, y


In [3]:
import time
from .src.FairCutForest import FairCutForest

In [8]:
datasets = [
    load_dataset('data/pendigits.mat')
]
k_planes = [1, 5]
modeldata = []

In [9]:

for X, y in datasets:
    display(X.shape)
    
    ext_lvls = [0, 1, 5, 'full']
    
    ext_lvls = [0, 1, 5, 'full']
    if X.shape[1] == 33:
        ext_lvls.append(10)
        ext_lvls.append(15)
    elif X.shape[1] == 16:
        ext_lvls.append(10)
    
    ks=[]
    es=[]
    rocaucs=[]
    praucs =[]
    times=[] 
    
    for e in ext_lvls:
        for k in k_planes:
            for i in range(5):
                print()
                print(e, k, i)
                model = FairCutForest(100, 256, k, e)

                start = time.time()
                model.fit(X)
                end = time.time()
                print('done')

                y_pred = model.predict(X)   
                np.stack((y, y_pred), axis=1)
                results = pd.DataFrame(np.stack((y, y_pred), axis=1))
                results.columns = ['actual', 'predicted']
                rocauc = '%.3f'% metrics.roc_auc_score(results.actual, results.predicted)

                precision, recall, thresholds = metrics.precision_recall_curve(results.actual, results.predicted)
                prauc = '%.3f'% metrics.auc(recall, precision)

                print(rocauc, prauc, '%.3f'% (end-start))

                times.append('%.3f'% (end-start))
                rocaucs.append(rocauc)
                praucs.append(prauc)
                ks.append(k)
                es.append(e)
            
    data.append((ks, es, rocaucs, praucs, times))
modeldata.append(data)

(6870, 16)


0 1 0
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||done
0.959 0.233 10.233

0 1 1
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||done
0.961 0.220 9.956

0 1 2
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||done
0.959 0.248 10.024

0 1 3
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||done
0.961 0.246 9.902

0 1 4
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||done
0.949 0.213 9.792

0 5 0
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||done
0.972 0.298 43.248

0 5 1
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||done
0.949 0.196 42.886

0 5 2
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [23]:
ann = data[0]
ks1, es1, rocaucs1, praucs1, times1 = ann

In [13]:
res = pd.DataFrame({'q' : ks1, 'e': es1, 'ROC AUC': rocaucs1, 'PR AUC': praucs1, 'Time [s]': times1})
res[['ROC AUC', 'PR AUC', 'Time [s]']] = res[['ROC AUC', 'PR AUC', 'Time [s]']].apply(pd.to_numeric)
c = res
ann = c.groupby(['q', 'e'])[['ROC AUC', 'PR AUC', 'Time [s]']].agg([np.mean])
c

,q,e,ROC AUC,PR AUC,Time [s]
0,1,0,0.894,0.393,10.565
1,1,0,0.886,0.376,10.493
2,1,0,0.904,0.406,10.287
3,1,0,0.903,0.437,10.509
4,1,0,0.894,0.348,11.240
5,5,0,0.869,0.321,46.903
6,5,0,0.897,0.420,47.544
7,5,0,0.894,0.415,45.384
8,5,0,0.894,0.382,45.597
9,5,0,0.889,0.420,47.109
